# GUI - Nine Men Morris
In diesem Abschnitt soll die graphische Oberfläche des Spiels entwickelt werden. Dies beinhaltet die Interaktion des Spielers mit dem Spielaufbau.

In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

## Mögliche Positionen
Die Grafiken unten zeigen zwei unterschiedliche Notationen für die Position der Spielsteine. Rechts ist die offizielle Notation des Welt-Mühle-Dachverbands (WMD) abgebildet, während links die interne Notation zu sehen ist.

### WMD-Notation
"Die Spalten werden von links beginnend von a bis g benannt. Die Zeilen werden von unten beginnend von 1 bis 7 durchnummeriert. Gültige Feldnamen sind beispielsweise a1, d2, e3 oder g7.

Bei dieser Notation gehört nicht zu jeder Buchstaben- und Zahlenkombination aus a..g und 1..7 auch ein Feld. Ungültige Feldbezeichnungen sind beispielsweise b3, d4, f5 oder g6. Diese Notation ähnelt der beim Schach, wo die 64 Felder von a..h und 1..8 benannt werden. Allerdings gehört beim Schach zu jeder Zahlen/Buchstabenkombination auch ein Feld.

Beim Setzen schreibt man das Zielfeld auf, zum Beispiel a1 oder e4. Beim Ziehen werden die Namen des durch Minus getrennten Quell- und Zielfeldes verwendet, zum Beispiel f2-f4 oder d3-d2. Wird ein Mühle gebildet und ein Stein geschlagen, dann wird der geschlagene Stein durch x getrennt und an den Zug angefügt, beispielweise f2-f4xd5 (Zug von f2 nach f4, Mühle und geschlagen wird d5)." - Dr. Rainer Rosenberger, http://muehlespieler.de/download/muehle_lehrbuch.pdf

### Interne Notation
Intern werden die Positionen als Tupel bestehend aus Ringnummer und Zellnummer beschrieben. Die Ringe, in der Abbildung farbig markiert, sind dabei von außen beginnend mit 0, 1 und 2 nummeriert. Für die Bezeichnung der Zellen wird in jedem Ring oben links mit 0 begonnen und im Uhrzeigersinn bis 7 weitergezählt. So ergibt sich eine eindeutige Notation.

<img src="board_positions.png" alt="Mögliche Positionen" width="800"/>

## Definitionen
Zunächst werden einige Definitionen für die gesamte Implementation festgesetzt: 
- ring = eines der 3 Quadrate (Wertebereich: 0-2)
- cell = ein Punkt auf einem Ring (Wertebereich: 0-7)
- position = Tupel: (ring, cell)
- board = Liste aus 3 Listen, die angeben, ob, und wenn ja welcher Stein dort sitzt. Dabei symbolisiert jede Liste einen Ring. In jeder Liste sind neun Werte. Je einer für eine Zelle im Ring. Eine 0 sagt dabei aus, dass an der entsprechenden Position kein Stein ist. Eine 1 oder 2 stehen für einen Stein des zugehörigen Spielers.
- remaining = \[Anzahl noch nicht gesetzter Steine Spieler 1, Anzahl noch nicht gesetzte Steine Spieler 2\]
- state = \[remaining, board\]

## Importe
Zur Visualisierung des Spielbrettes wird [ipycanvas](https://ipycanvas.readthedocs.io/en/latest/index.html) verwendet.

In [ ]:
%run ./Muehle_Logic.ipynb
#from __future__ import print_function
import ipycanvas
from ipycanvas import MultiCanvas

#from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## Konstanten

* BOARD_SIZE = Größe des Spielfeldes in Pixeln
* DOT_RADIUS = Radius der schwarzen kleinen Punkte, die mögliche Positionen markieren (in Abhängigkeit von der Spielfeldgröße)
* PIECE_RADIUS = Radius der Spielsteine
* COLOUR = Farben der [dots, pieces_player_1, pieces_player_2]
* COLOUR_HINT = Farbe, in der mögliche Zielfelder markiert werden
* COLOUR_OPPONENT = Farbe, in der entfernbare Steine markiert werden
* PADDING = relativer Abstand des äußersten Quadrats zum Spielfeldrand
* DISTANCE = relativer Abstand zwischen den Quadraten des Spielfelds
* TRANSPARENCY_DEFAULT = Standardwert für die Transparenz
* TRANSPARENCY_HINT = Transparenz der Kreise möglicher Zielfelder und entfernbarer Steine

In [ ]:
BOARD_SIZE      = 400
DOT_RADIUS      = BOARD_SIZE*0.025
PIECE_RADIUS    = BOARD_SIZE*0.04
COLOUR          = ['black', 'white', 'sienna']
COLOUR_HINT     = 'green'
COLOUR_OPPONENT = 'red'
PADDING         = 0.05
DISTANCE        = 0.15
TRANSPARENCY_DEFAULT = 1.0
TRANSPARENCY_HINT    = 0.5

 `row()` liefert einen relativen Wert für die Position der eingegebenen Reihe. `col()` macht dies für die Spalten.

In [ ]:
#Reihe von oben nach unten (0-6)
def row(number):
    return PADDING + DISTANCE * number
#Spalte von links nach rechts (0-6)
def col(number):
    return row(number)

`positions` beschreibt die möglichen Punkte, an denen Steine sitzen können.  `wmd_dic` überführt die Positionen in die Schreibweise des WMD.

In [ ]:
positions = [([col(0), row(0)], [col(3), row(0)], [col(6), row(0)], [col(6), row(3)], [col(6), row(6)], [col(3), row(6)], [col(0), row(6)], [col(0), row(3)]), #ring 0
             ([col(1), row(1)], [col(3), row(1)], [col(5), row(1)], [col(5), row(3)], [col(5), row(5)], [col(3), row(5)], [col(1), row(5)], [col(1), row(3)]), #ring 1
             ([col(2), row(2)], [col(3), row(2)], [col(4), row(2)], [col(4), row(3)], [col(4), row(4)], [col(3), row(4)], [col(2), row(4)], [col(2), row(3)])] #ring 2

wmd_dic = {
    "a1":positions[0][6],
    "a4":positions[0][7],
    "a7":positions[0][0],
    "b2":positions[1][6],
    "b4":positions[1][7],
    "b6":positions[1][0],
    "c3":positions[2][6],
    "c4":positions[2][7],
    "c5":positions[2][0],
    "d1":positions[0][5],
    "d2":positions[1][5],
    "d3":positions[2][5],
    "d5":positions[2][1],
    "d6":positions[1][1],
    "d7":positions[0][1],
    "e3":positions[2][4],
    "e4":positions[2][3],
    "e5":positions[2][2],
    "f2":positions[1][4],
    "f4":positions[1][3],
    "f6":positions[1][2],
    "g1":positions[0][4],
    "g4":positions[0][3],
    "g7":positions[0][2]
}

## Initialisierung Canvas
Im folgenden wird der Aufbau des Spielfeldes erstellt. Dies erfolgt mit Hilfe von ipycanvas. Das Spielbrett (`board`) besteht aus drei übereinanderliegenden Ebenen. Ebene 0 ist der Hintergrund, ein beiges Quadrat. Ebene 1 beinhaltet schwarzen die Striche und Punkte auf dem Spielfeld. Diese setzen sich aus drei schwarzen Quadraten, viel horizontalen bzw. waagrechten Linien und 24 Punkten, die die möglichen Positionen der Steine markieren, zusammen. Dabei wird bei der Erstellung mit relativen Positionsangaben gearbeitet, um das Spielfeld einfach skalieren zu können. In der Ebene 2 liegen schließlich die Spielsteine. 

In [ ]:
def init_canvas():
    #board[Hintergrund, Linien, Steine]
    board = MultiCanvas(3, width = BOARD_SIZE, height = BOARD_SIZE)

    # Hintergrund
    board[0].fill_style = '#ffffcc'
    board[0].fill_rect(0, 0, BOARD_SIZE)

    # Strichstärke
    board[1].line_width = 5

    # Quadrate
    board[1].stroke_rect(BOARD_SIZE*col(0), BOARD_SIZE*row(0), BOARD_SIZE*(1-row(0)-col(0))) #ring 0
    board[1].stroke_rect(BOARD_SIZE*col(1), BOARD_SIZE*row(1), BOARD_SIZE*(1-row(1)-col(1))) #ring 1
    board[1].stroke_rect(BOARD_SIZE*col(2), BOARD_SIZE*row(2), BOARD_SIZE*(1-row(2)-col(2))) #ring 2

    # Mittelinien
    board[1].begin_path()
    board[1].move_to(BOARD_SIZE*col(3), BOARD_SIZE*row(0)) #oben
    board[1].line_to(BOARD_SIZE*col(3), BOARD_SIZE*row(2))
    board[1].move_to(BOARD_SIZE*col(6), BOARD_SIZE*row(3)) #rechts
    board[1].line_to(BOARD_SIZE*col(4), BOARD_SIZE*row(3))
    board[1].move_to(BOARD_SIZE*col(3), BOARD_SIZE*row(6)) #unten
    board[1].line_to(BOARD_SIZE*col(3), BOARD_SIZE*row(4))
    board[1].move_to(BOARD_SIZE*col(0), BOARD_SIZE*row(3)) #links
    board[1].line_to(BOARD_SIZE*col(2), BOARD_SIZE*row(3))
    board[1].stroke()

    # Punkte (außen, mitte, innen)
    for ring in positions:
        for x,y in ring:
            board[1].fill_arc(BOARD_SIZE*x, BOARD_SIZE*y, DOT_RADIUS, 0, 360)
    return board

In [ ]:
board = init_canvas()

## Status des Spielfelds
Der Status speichert die aktuelle Situation des Spielfeldes. Die Klasse beinhaltet mehrere Variablen. `state` gibt wie in der Definition beschrieben an, ob und wenn ja welcher Stein sich an einer Position auf dem Spielfeld befindet. Dabei gilt:
* 0 = kein Stein
* 1 = weißer Stein
* 2 = brauner Stein

Außerdem beinhaltet die Variable Angaben über die Steine, die die beiden Spieler nocht setzen können.
`current_player` gibt an, welcher Spieler, 1 oder 2, gerade am Zug ist. `mills` ist die Menge aller aktuell bestehenden Mühlen. In `number_stones_to_remove` wird gespeichert, wie viele Steine der aktuelle Spieler aufgrund von Mühlen noch entfernen darf. `selected_dot_old` und `selected_dot_new` sind jeweils ein Tupel, der die im letzten bzw. diesem Zug geklickte Position speichert. Dieses Tupel haben die Form `(ring, cell)`. Dies ist notwendig für Zug- und Springvorgänge. Dabei ist vor allem in Phase 2 des Spiels relevant, woher der Stein stammt, um valide Zielfelder zu ermitteln. `move_or_jump` gibt an, ob sich der Spieler gerade in einer Zug- oder Sprungaktion befindet. Ist die Variable True, so hat der Spieler bereits den zu versetzenden Stein ausgewählt und muss noch auf das Zielfeld klicken. `game_over`ist False, solange das Spiel noch läuft.

**@Julian** In welchen Format wird `mills` gespeichert?

In [ ]:
class Status():
    state = [[9,9],[          # Anzahl zu setzender Steine (Spieler_1 (Weiß), Spieler_2 (Braun))
            [0,0,0,0,0,0,0,0], # ring 0
            [0,0,0,0,0,0,0,0], # ring 1
            [0,0,0,0,0,0,0,0]  # ring 2
      ]]
    current_player = 1
    mills = set()              # Tupel der Form (Spieler, Ring, Zelle)
    number_stones_to_remove = 0
    selected_dot_old = (None, None)
    selected_dot_new = (None, None)
    move_or_jump = False
    game_over = False

## Initialisierung und Funktionen der Status-Widgets und des Restart-Button

Die Methode `on_button_clicked(b)` wird durch den restart_button aufgerufen und setzt den Status des Spieles wieder auf den Startzustand.

In [ ]:
def on_button_clicked(b):
    Status.state= [[9,9],[          
                    [0,0,0,0,0,0,0,0], 
                    [0,0,0,0,0,0,0,0], 
                    [0,0,0,0,0,0,0,0]  
                   ]]
    Status.current_player = 1
    Status.mills = set()
    Status.number_stones_to_remove = 0
    Status.selected_dot_old = (None, None)
    Status.selected_dot_new = (None, None)
    Status.move_or_jump = False
    Status.game_over = False
    update_board(Status)

An dieser Stelle findet die Initialisierung der unterschiedlichen Statuselemente statt.
-  Das turn_label ist ein Label, das den Spieler anzeigt, der aktuell an der Reihe ist
-  Das status_label ist für weitere Statusanzeigen vorgesehen, wie die Aufforderung an den aktuellen Spieler, im Fall einer Mühle, einen Stein des Gegners zu entfernen.
-  Das pieces_status_label zeigt je nach Spielphase entweder die Anzahl der Steine, die noch gesetzt werden können (Phase 1), oder die noch übrig sind (Phase 2/3). 
-  Der restart_button ruft die Methode `on_button_clicked(b)` auf und setzt somit das Spiel auf den Startzustand zurück
-  stones_player_one und stones_player_two zeigen jeweils einen horizontalen Balken, der in Phase 1 nach jedem gesetzten Stein kleiner wird, bis keiner mehr vorhanden ist. In Phase 2 und 3 ist die Auslenkung des Balkens von der Anzahl der Steine des jeweiligen Spielers abhängig. Somit ist schnell erkennbar, welcher Spieler gerade vorne liegt.


In [ ]:
turn_label = widgets.Label(value='Spieler x ist an der Reihe')

status_label = widgets.Label(value='aktueller Status:')

pieces_status_label = widgets.Label(value='Setzbare Steine:')

restart_button = widgets.Button(description='restart')
restart_button.on_click(on_button_clicked)

stones_player_one = widgets.FloatProgress(
                value=9,
                max=9,
                min=0,
                description='Spieler 1',
                bar_style= 'info', # 'success', 'info', 'warning', 'danger' or ''
                orientaiton='horizontal'
            )

stones_player_two = widgets.FloatProgress(
                value=9,
                max=9,
                min=0,
                description='Spieler 2',
                bar_style='warning', # 'success', 'info', 'warning', 'danger' or ''
                orientaiton='horizontal'
            )

Die Methode `change_status_label(text)` nimmt einen beliebigen Status-Text entgegen und setzt diesen mit dem Präfix 'Status: ' als Text des status_label.

In [ ]:
def change_status_label(text):
    status_label.value = 'Status: '+text

Die Methode `update_status_widgets()` aktualisiert die unterschiedlichen Statusanzeigen passend zur aktuellen Spielphase.

In [ ]:
def update_status_widgets():
    turn_label.value='Spieler '+ str(Status.current_player)+ ' ist dran.'
    change_status_label('Spieler in Phase '+ str(get_player_phase(Status.state, Status.current_player))+'.')
    if(Status.state[0][0] == Status.state[0][1] == 0):
        pieces_status_label.value = 'Verbleibende Steine:'
        stones_player_one.value = len(player_pieces(Status.state, 1))
        stones_player_two.value = len(player_pieces(Status.state, 2))
    else:
        pieces_status_label.value = 'Setzbare Steine:'
        stones_player_one.value = Status.state[0][0]
        stones_player_two.value = Status.state[0][1]

## Spielsteine anzeigen

Die Funktion `draw_piece()` zeichnet an einer zu übergebenden Position einen Spielstein für den eingegebenen Spieler. Dieser besteht aus einem mit der Spielerfarbe gefüllten Kreis als Hintergrund und zwei Innenringen um die Steine realistischer aussehen zu lassen.

In [ ]:
def draw_piece(ring, cell, player):
    #Hintergrund
    board[2].fill_style = COLOUR[player]
    board[2].fill_arc(BOARD_SIZE*positions[ring][cell][0], BOARD_SIZE*positions[ring][cell][1], PIECE_RADIUS, 0, 360)
    #Ringe
    if player == 1:
        board[2].stroke_style = 'silver' 
    else:
        board[2].stroke_style = 'chocolate'
    board[2].stroke_arc(BOARD_SIZE*positions[ring][cell][0], BOARD_SIZE*positions[ring][cell][1], PIECE_RADIUS,     0, 360)
    board[2].stroke_arc(BOARD_SIZE*positions[ring][cell][0], BOARD_SIZE*positions[ring][cell][1], PIECE_RADIUS*0.7, 0, 360)
    board[2].stroke_arc(BOARD_SIZE*positions[ring][cell][0], BOARD_SIZE*positions[ring][cell][1], PIECE_RADIUS*0.3, 0, 360)

`update_board()` zeichnet die obere Ebene des Spielbretts nach jeder Veränderung neu. So werden Positionsänderungen oder Hinweise zu möglichen Zielfeldern und schlagbaren Steinen sichtbar. Zusätzlich wird hier die Aktualisierung der Statusanzeige angestoßen.

In [ ]:
def update_board(status):
    one_below_other = widgets.VBox([turn_label, status_label, pieces_status_label, stones_player_one, stones_player_two, restart_button])
    side_by_side = widgets.HBox([board, one_below_other])
    display(side_by_side)

    with ipycanvas.hold_canvas(board):
        board[2].clear()
        board[2].global_alpha = TRANSPARENCY_DEFAULT
        for ring in range(3):
            for cell in range(8):
                player = status.state[1][ring][cell]
                if player in [1, 2]: draw_piece(ring, cell, player)
        update_status_widgets()
    #return board

Die Funktion `on_dot()` überprüft, ob die übergebenen x- und y- Koordinaten in der übergebenen Position liegen und gibt einen boolschen Wert zurück.

In [ ]:
def on_dot(pos, x, y):
    return (pos[0]*BOARD_SIZE-PIECE_RADIUS/2 < x < pos[0]*BOARD_SIZE+PIECE_RADIUS/2 and pos[1]*BOARD_SIZE-PIECE_RADIUS/2 < y < pos[1]*BOARD_SIZE+PIECE_RADIUS/2)

Mit Hilfe von `highlight_positions()` werden alle Positionen aus einer Liste mit einem transparenten Kreis markiert. Falls keine andere Farbe übergeben wird, wird die in den Konstanten definierte default-Farbe verwendet.

In [ ]:
def highlight_positions(highlight_positions, colour = COLOUR_HINT):
    with ipycanvas.hold_canvas(board):
        for position in highlight_positions:
            ring, cell = position
            board[2].fill_style = colour
            board[2].global_alpha = TRANSPARENCY_HINT
            board[2].fill_arc(BOARD_SIZE*positions[ring][cell][0], BOARD_SIZE*positions[ring][cell][1], PIECE_RADIUS, 0, 360)

Bei jedem Mausklick auf das Spielfeld wird `handle_mouse_down()` mit den Koordinaten des Klicks aufgerufen. Wenn das Spiel noch nicht vorbei ist, wir dann überprüft, ob der Klick an einer Positionen, also im Bereich um einen schwarzen Punkt, aufgerufen wurde. Ist dies der Fall, wird die Position gespeichert und die Funktion `make_move()` aus dem Logik-Abschnitt aufgerufen.

In [ ]:
def handle_mouse_down(x, y):
    global Status
    if not Status.game_over:
        for ring in positions:
            for cell in ring:
                if on_dot(cell, x, y):
                    Status.selected_dot_old = Status.selected_dot_new
                    Status.selected_dot_new = (positions.index(ring), ring.index(cell))
                    try:
                        make_move(positions.index(ring), ring.index(cell))
                    except InvalidMoveException:
                        print('Ungültiger Zug')
board[2].on_mouse_down(handle_mouse_down)

Schließlich kann das Spielfeld erzeugt und ein Spiel gestartet werden.

In [ ]:
update_board(Status)